In [1]:
import os
import math
import sys
# Assuming a workspace with:
#   src/tracetools_analysis/
#   src/ros-tracing/ros2_tracing/tracetools_read/
sys.path.insert(0, '../')
sys.path.insert(0, '../../../ros-tracing/ros2_tracing/tracetools_read/')
import datetime as dt

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tracetools_analysis.loading import load_file
from tracetools_analysis.processor.ros2 import Ros2Handler
from tracetools_analysis.utils.ros2 import Ros2DataModelUtil

In [2]:
#path = '/home/user/pnn16004/ros2_ws/tracing/ros2/'
path = '/home/user/pnn16004/ros2_ws/tracing/ros2_3/'
#f.name, f.path
folders = [ f for f in os.scandir(path) if f.is_dir() ]

In [ ]:
for folder in folders:
    if folder.name == "ros2_data" or folder.name == "read":
        continue
    subfolders = [ f for f in os.scandir(folder.path) if f.is_dir() ]
    for subfolder in subfolders:
        # Process
        events = load_file(subfolder.path + '/ust')
        handler = Ros2Handler.process(events)
        #handler.data.print_data()

        data_util = Ros2DataModelUtil(handler.data)
        try:
            callback_symbols = data_util.get_callback_symbols()
        except:
            print("Callback error!")
            continue
            pass

        # Plot durations separately
        for obj, symbol in callback_symbols.items():
            owner_info = data_util.get_callback_owner_info(obj)
            if owner_info is None:
                owner_info = '[unknown]'

            # Filter out internal subscriptions
            if '/parameter_events' in owner_info:
                continue

            if "/mros/ping" in owner_info:
                color = '#29788E'
                name = 'Ping'
            elif "/mros/pong" in owner_info:
                color = '#DD4968'
                name = 'Pong'
            elif "/mros/peng" in owner_info:
                color = '#410967'
                name = 'Peng'
            else:
                continue
                
            # Duration
            duration_df = data_util.get_callback_durations(obj)

            source = pd.DataFrame(duration_df)

            timestamp = source['timestamp'].apply(lambda x: x.value)
            duration = source['duration'].apply(lambda x: x.value)

            time = np.array(timestamp.values)
            data = np.array(duration.values)

            time = (time - time[0]) / pow(10, 9)
            data = data / pow(10, 6)
            
            combined = np.stack([time, data], axis=1)
            
            saveFolder = path + "ros2_data/" + folder.name + "/"
            try:
                os.makedirs(saveFolder)
                print("Created folder: " + saveFolder)
            except OSError as error:
                pass
                
            np.savetxt(saveFolder + name + ".csv", combined, delimiter=",")

found converted file: /home/user/pnn16004/ros2_ws/tracing/ros2_3/32s-due-100-3/test_pong_trace/ust/converted
 [100%] [Ros2Handler]
Callback error!
found converted file: /home/user/pnn16004/ros2_ws/tracing/ros2_3/32s-due-100-3/test_ping_trace/ust/converted
